## API EDA PROCESS

Luis Angel Garcia (2230177)

Libraries

In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

External configuration load

In [8]:
def leer_configuracion_spotify(ruta_archivo):
    config = {}
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            if '=' in linea:
                clave, valor = linea.strip().split('=', 1)
                config[clave.strip()] = valor.strip()
    return config

config = leer_configuracion_spotify("../config_documents/spotify_config.txt")

read artist from spotify_api_artist

In [12]:
artistas_colombianos = []
with open("../config_documents/spotify_api_artist.txt", 'r', encoding='utf-8') as archivo:
    for linea in archivo:
        if linea.strip():
            artistas_colombianos.append(linea.strip())

User autentification

In [13]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=config['CLIENT_ID'],
    client_secret=config['CLIENT_SECRET']
))

search artist from api

In [14]:
artistas_data = []
for nombre in artistas_colombianos:
    resultado = sp.search(q=f"artist:{nombre}", type='artist', limit=1)
    if resultado['artists']['items']:
        artista = resultado['artists']['items'][0]
        artistas_data.append({
            'artist_id': artista['id'],
            'artist_name': artista['name'],
            'popularity': artista['popularity'],
            'followers': artista['followers']['total'],
            'genres': ', '.join(artista['genres'])
        })

DF with user TOP artist

In [16]:
df = pd.DataFrame(artistas_data)
df

,artist_id,artist_name,popularity,followers,genres
0,0EmeFodog0BfCgMzAIvKQp,Shakira,89,37520825,"latin pop, latin"
1,790FomKkXshlbRYZFtlgla,KAROL G,89,55979455,"reggaeton, latin, urbano latino"
2,1r4hJ1h58CWwUQe3MxPuau,Maluma,87,36558154,"reggaeton, latin, urbano latino"
3,1vyhD5VmyZ7KMfW5gqLgo5,J Balvin,89,38154384,"reggaeton, latin"
4,07YUOmWljBTXwIseAUd9TW,Sebastian Yatra,81,23814886,"colombian pop, reggaeton, latin"
5,0tmwSHipWxN12fsoLcFU3B,Manuel Turizo,83,14913273,"reggaeton, latin, colombian pop"
6,28gNT5KBp7IjEOQoevXf9N,Camilo,83,20021057,colombian pop
7,2LRoIwlKmHjgvigdNGBHNo,Feid,90,16848947,"reggaeton, urbano latino"
8,53KTldaJ8tHSkYU3nigfwP,Fonseca,71,3321028,"colombian pop, latin pop"
9,49Z1AvGeUaBSanPaOmplK6,Andrés Cepeda,65,3624447,"colombian pop, latin pop"
